In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
import warnings
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')

### Load data

In [2]:
train_path = '../../hy_round1_train_20200102'
test_path = '../../hy_round1_testB_20200221'
train_files = os.listdir(train_path)
test_files = os.listdir(test_path)

In [3]:
ret = []
for file in tqdm(train_files):
    df = pd.read_csv(f'{train_path}/{file}')
    ret.append(df)
df_train = pd.concat(ret)
df_train.columns = ['ship','x','y','v','d','time','type']

100%|██████████████████████████████████████████████████████████████████| 7000/7000 [04:39<00:00, 29.86it/s]


In [4]:
ret = []
for file in tqdm(test_files):
    df = pd.read_csv(f'{test_path}/{file}')
    ret.append(df)
df_test = pd.concat(ret)
df_test.columns = ['ship','x','y','v','d','time']

100%|██████████████████████████████████████████████████████████████████| 2000/2000 [00:38<00:00, 52.30it/s]


### Feature extract

In [5]:
def group_feature(df, key, target, aggs):   
    agg_dict = {}
    for ag in aggs:
        agg_dict[f'{target}_{ag}'] = ag
    print(agg_dict)
    t = df.groupby(key)[target].agg(agg_dict).reset_index()
    return t

def extract_feature(df, train,test_model=False):
    # 基本 统计特征
    t = group_feature(df, 'ship','x',['max','min','mean','std','skew','sum'])
    train = pd.merge(train, t, on='ship', how='left')
    t = group_feature(df, 'ship','x',['count'])
    train = pd.merge(train, t, on='ship', how='left')
    t = group_feature(df, 'ship','y',['max','min','mean','std','skew','sum'])
    train = pd.merge(train, t, on='ship', how='left')
    t = group_feature(df, 'ship','v',['max','min','mean','std','skew','sum'])
    train = pd.merge(train, t, on='ship', how='left')
    t = group_feature(df, 'ship','d',['max','min','mean','std','skew','sum'])
    train = pd.merge(train, t, on='ship', how='left')

    # 上下1/4分位数，x y的协方差，相关系数
    t=df.groupby('ship')['x'].agg({'x_25':lambda x: np.percentile(x,25),'x_50':lambda x: np.percentile(x,50)
                                   ,'x_75':lambda x: np.percentile(x,75)})
    train = pd.merge(train, t, on='ship', how='left')
    t=df.groupby('ship')['y'].agg({'y_25':lambda x: np.percentile(x,25),'y_50':lambda x: np.percentile(x,50),
                                   'y_75':lambda x: np.percentile(x,75)})
    train = pd.merge(train, t, on='ship', how='left')
    t=df.groupby('ship')['v'].agg({'v_25':lambda x: np.percentile(x,25),'v_50':lambda x: np.percentile(x,50),
                                   'v_75':lambda x: np.percentile(x,75)})
    train = pd.merge(train, t, on='ship', how='left')
    t=df.groupby('ship')['d'].agg({'d_25':lambda x: np.percentile(x,25),'d_50':lambda x: np.percentile(x,50),
                                   'd_75':lambda x: np.percentile(x,75)})
    train = pd.merge(train, t, on='ship', how='left')
    train['xy_cov']=df[['ship','x','y']].groupby('ship').cov().values[::2,1]
    train['xy_corr']=df[['ship','x','y']].groupby('ship').corr().values[::2,1]
    
    # x,y 交叉特征
    train['x_max_x_min'] = train['x_max'] - train['x_min']
    train['y_max_y_min'] = train['y_max'] - train['y_min']
    train['y_max_x_min'] = train['y_max'] - train['x_min']
    train['x_max_y_min'] = train['x_max'] - train['y_min']
    train['slope'] = train['y_max_y_min'] / np.where(train['x_max_x_min']==0, 0.001, train['x_max_x_min'])
    train['area'] = train['x_max_x_min'] * train['y_max_y_min']
    
    mode_hour = df.groupby('ship')['hour'].agg(lambda x:x.value_counts().index[0]).to_dict()
    train['mode_hour'] = train['ship'].map(mode_hour)
    
    # 时间特征
    t = group_feature(df, 'ship','hour',['max','min'])
    train = pd.merge(train, t, on='ship', how='left')
    
    hour_nunique = df.groupby('ship')['hour'].nunique().to_dict()
    date_nunique = df.groupby('ship')['date'].nunique().to_dict()
    train['hour_nunique'] = train['ship'].map(hour_nunique)
    train['date_nunique'] = train['ship'].map(date_nunique)

    t = df.groupby('ship')['time'].agg({'dif_time':lambda x:np.max(x)-np.min(x)}).reset_index()
    t['dif_day'] = t['dif_time'].dt.days
    t['dif_second'] = t['dif_time'].dt.seconds
    train = pd.merge(train, t, on='ship', how='left')
    return train

def extract_dt(df):
    df['time'] = pd.to_datetime(df['time'], format='%m%d %H:%M:%S')
#     df['month'] = df['time'].dt.month
    df['day'] = df['time'].dt.day
    df['date'] = df['time'].dt.date
    df['hour'] = df['time'].dt.hour
    
    return df

In [6]:
train_label = df_train.drop_duplicates('ship')
test_label = df_test.drop_duplicates('ship')

In [7]:
train_label.head()

ship             x             y     v    d           time type
0     0  6.152038e+06  5.124873e+06  2.59  102  1110 11:58:19   拖网
0     1  6.076254e+06  5.061743e+06  3.99  278  1110 11:40:21   拖网
0    10  6.321032e+06  5.242805e+06  4.48  213  1110 11:49:36   拖网
0   100  6.102751e+06  5.112534e+06  0.00    0  1030 23:50:05   拖网
0  1000  6.843713e+06  5.480538e+06  2.00  216  1106 23:42:30   围网

In [8]:
train_label['type'].value_counts(1)

拖网    0.623000
围网    0.231571
刺网    0.145429
Name: type, dtype: float64

In [9]:
type_map = dict(zip(train_label['type'].unique(), np.arange(3)))
type_map_rev = {v:k for k,v in type_map.items()}
train_label['type'] = train_label['type'].map(type_map)

In [10]:
train_label =  extract_feature(extract_dt(df_train), train_label) 

{'x_max': 'max', 'x_min': 'min', 'x_mean': 'mean', 'x_std': 'std', 'x_skew': 'skew', 'x_sum': 'sum'}
{'x_count': 'count'}
{'y_max': 'max', 'y_min': 'min', 'y_mean': 'mean', 'y_std': 'std', 'y_skew': 'skew', 'y_sum': 'sum'}
{'v_max': 'max', 'v_min': 'min', 'v_mean': 'mean', 'v_std': 'std', 'v_skew': 'skew', 'v_sum': 'sum'}
{'d_max': 'max', 'd_min': 'min', 'd_mean': 'mean', 'd_std': 'std', 'd_skew': 'skew', 'd_sum': 'sum'}
{'hour_max': 'max', 'hour_min': 'min'}


In [11]:
test_label =  extract_feature(extract_dt(df_test), test_label,test_model=True) 

{'x_max': 'max', 'x_min': 'min', 'x_mean': 'mean', 'x_std': 'std', 'x_skew': 'skew', 'x_sum': 'sum'}
{'x_count': 'count'}
{'y_max': 'max', 'y_min': 'min', 'y_mean': 'mean', 'y_std': 'std', 'y_skew': 'skew', 'y_sum': 'sum'}
{'v_max': 'max', 'v_min': 'min', 'v_mean': 'mean', 'v_std': 'std', 'v_skew': 'skew', 'v_sum': 'sum'}
{'d_max': 'max', 'd_min': 'min', 'd_mean': 'mean', 'd_std': 'std', 'd_skew': 'skew', 'd_sum': 'sum'}
{'hour_max': 'max', 'hour_min': 'min'}


In [12]:
features = [x for x in train_label.columns if x not in ['ship','type','time','dif_time','date',
                                                       'dif_day','x_50','d_skew','x_skew','date_nunique','v_skew','v_50','x_count',
                                                       'y_50','x_max_x_min','d_50','y_max_y_min','hour_nunique','y_skew','hour_min',
                                                       'mode_hour','xy_corr','hour_max','month','d_min','weekday',
                                                        'angle','hour','v_min','d_max','d_25'
                                                       ]]
target = 'type'

In [13]:
print(len(features),len(train_label.columns), ','.join(features))

34 60 x,y,v,d,x_max,x_min,x_mean,x_std,x_sum,y_max,y_min,y_mean,y_std,y_sum,v_max,v_mean,v_std,v_sum,d_mean,d_std,d_sum,x_25,x_75,y_25,y_75,v_25,v_75,d_75,xy_cov,y_max_x_min,x_max_y_min,slope,area,dif_second


### How to train? 

- Lightgbm 单模 
- 总共提取了60个特征，丢弃一些不重要特征，最终34个特征用于训练
- Way: [OOF](https://stackoverflow.com/questions/52396191/what-is-oof-approach-in-machine-learning).10 folds cross-validation

In [14]:
lgb_params = {
    'learning_rate' : 0.1,
    'n_estimators': 1000,
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'num_class': 3,
    'early_stopping_rounds': 100,
}

In [15]:
fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
X = train_label[features].copy()
y = train_label[target]
X_test=test_label[features].copy()
models = []
pred = np.zeros((len(test_label),3))
oof = np.zeros((len(X), 3))

for index, (train_idx, val_idx) in enumerate(fold.split(X, y)):

    train_set = lgb.Dataset(X.iloc[train_idx], y.iloc[train_idx])
    val_set = lgb.Dataset(X.iloc[val_idx], y.iloc[val_idx])

    model = lgb.train(lgb_params, train_set, valid_sets=[train_set, val_set], verbose_eval=100)
    models.append(model)
    val_pred = model.predict(X.iloc[val_idx])
    oof[val_idx] = val_pred
    val_y = y[val_idx]
    val_pred = np.argmax(val_pred, axis=1)
    print(index, 'val f1', metrics.f1_score(val_y, val_pred, average='macro'))

    test_pred = model.predict(X_test)
    pred += test_pred/10

Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 0.0807159	valid_1's multi_logloss: 0.255133
[200]	training's multi_logloss: 0.0216962	valid_1's multi_logloss: 0.25519
Early stopping, best iteration is:
[132]	training's multi_logloss: 0.0522428	valid_1's multi_logloss: 0.250054
0 val f1 0.8794052341956773
Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 0.0836319	valid_1's multi_logloss: 0.222797
[200]	training's multi_logloss: 0.0232902	valid_1's multi_logloss: 0.209302
[300]	training's multi_logloss: 0.00681649	valid_1's multi_logloss: 0.21673
Early stopping, best iteration is:
[249]	training's multi_logloss: 0.0125931	valid_1's multi_logloss: 0.206417
1 val f1 0.9066535134911664
Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 0.0800782	valid_1's multi_logloss: 0.239536
[200]	training's multi_logloss: 0.0224598	valid_1's multi_logloss: 0.230444
Early

In [16]:
oof = np.argmax(oof, axis=1)
print('oof f1', metrics.f1_score(oof, y, average='macro'))
# oof f1 0.887324896925613 10folds 0.8859582023865146 5 folds 

oof f1 0.887324896925613


### Write submission

In [17]:
pred = np.argmax(pred, axis=1)
sub = test_label[['ship']]
sub['pred'] = pred

print(sub['pred'].value_counts(1))
sub['pred'] = sub['pred'].map(type_map_rev)
sub.to_csv('result.csv', index=None, header=None)

0    0.6185
1    0.2365
2    0.1450
Name: pred, dtype: float64


### Features importance

In [57]:
ret = []
for index, model in enumerate(models):
    df = pd.DataFrame()
    df['name'] = model.feature_name()
    df['score'] = model.feature_importance()
    df['fold'] = index
    ret.append(df)
    
df = pd.concat(ret)

In [58]:
df = df.groupby('name', as_index=False)['score'].mean()
df = df.sort_values(['score'], ascending=False)

In [59]:
df

name  score
29  y_max_x_min  832.5
19  x_max_y_min  777.4
16         x_25  695.9
27         y_75  643.7
13        v_std  637.6
28        y_max  617.7
7         slope  612.6
22        x_std  606.9
10         v_75  576.0
6    dif_second  569.2
21        x_min  562.3
24       xy_cov  536.6
4         d_std  534.6
32        y_std  510.6
15            x  501.1
31        y_min  498.1
11        v_max  492.1
23        x_sum  490.9
3        d_mean  480.4
2          d_75  479.9
25            y  447.6
17         x_75  442.5
18        x_max  438.1
26         y_25  432.4
8             v  420.3
0          area  404.7
5         d_sum  404.7
33        y_sum  393.1
1             d  379.2
12       v_mean  371.3
9          v_25  370.3
14        v_sum  363.6
30       y_mean  333.9
20       x_mean  251.2